In [1]:
from uAI_lib import getData, loadDataFromCSV, timeStampToDate, saveCSV
from uAI_plot import plot_candleStick
from uAI_Finance import Finance
from uAI_typedef import *

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import pandas as pd
plotly.tools.set_credentials_file(username='jbernardoCruz', api_key='wLC85rHSk0rclw6QbkAn')

In [2]:
'''GET DATA'''

timeFrame = 15
coin = 'ETHUSD'
fileName = 'data_test.csv'

data = getData(str(timeFrame) + 'm', coin, 2500)
saveCSV(data, fileName)

Requesting Data: https://api.bitfinex.com/v2/candles/trade:15m:tETHUSD/hist?&limit=2500

Data successfully acquired

Saving .csv file...

File saved: data_test.csv



In [24]:
'''LOAD DATA'''

csvData = loadDataFromCSV(fileName, [TIME, OPEN, HIGH, LOW, CLOSE, VOL])
finance = Finance(csvData)
data = finance.data

#TIME
time = data[TIME]
for i in range(len(time)):
    time[i] = i
    
#RSI
rsi = finance.RSI()
rsiUp = []
rsiLow = []
rsi_aux = []
for i, value in enumerate(rsi[RSI]):
    rsiUp.append(70)
    rsiLow.append(30)
    if value > 30 and value < 70: pass
    else: rsi_aux.append(value)

#Stochastic RSI
s_rsi = finance.S_RSI()
s_rsi_k = s_rsi[S_RSI_K]
s_rsi_d = s_rsi[S_RSI_D]

# EMA
ema = finance.EMA(1)
ema50 = finance.EMA(50)
ema200 = finance.EMA(200)


Finance data set.


In [38]:
'''PLOT DATA'''

colors = ['rgb(67,67,67)', 'rgb(49,130,189)']

#Price
price_graph = go.Candlestick(x=data[TIME], open=data[OPEN], high=data[HIGH], low=data[LOW], close=data[CLOSE], name=str(coin))

#EMA
ema50_graph = go.Scatter(x = time, y = ema50,  name='EMA 50', line=dict(color=colors[0]))
ema200_graph = go.Scatter(x = time, y = ema200,  name='EMA 200', line=dict(color=colors[1]))

#RSI        
rsi_graph = go.Scatter(x = time, y = rsi,  name='RSI',  mode='lines', line=dict(color=colors[0]))
rsiUp_graph = go.Scatter(x = time, y = rsiUp, name='RSI 70', mode='lines', line=dict(color=colors[1]))
rsiLow_graph = go.Scatter(x = time, y = rsiLow, name='RSI 30', mode='lines', line=dict(color=colors[1]))

#Stochastic RSI
s_rsi_k_graph = go.Scatter(x = time, y = s_rsi_k,  name='Stochastic RSI')
s_rsi_d_graph = go.Scatter(x = time, y = s_rsi_d,  name='Stochastic RSI')

#Plot
fig = plotly.tools.make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(price_graph, 1, 1)
fig.append_trace(ema50_graph, 1, 1)
fig.append_trace(ema200_graph, 1, 1)
fig.append_trace(rsi_graph, 2, 1)
fig.append_trace(rsiUp_graph, 2, 1)
fig.append_trace(rsiLow_graph, 2, 1)

layout = go.Layout(xaxis = dict(rangeslider = dict(visible = False)),
                   yaxis1=dict(domain=[0.40, 1]), 
                   yaxis2=dict(domain=[0, 0.35]),
                   autosize=False, width=1000,height=800, title='uAI')
    
fig['layout'].update(layout)
              
py.iplot(fig, filename='simple_candlestick')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jbernardoCruz/0 or inside your plot.ly account where it is named 'simple_candlestick'


In [39]:
#Plot RSI Pulses
test = []
rsi_mat = rsi.as_matrix()

for i in range(len(rsi)):    
   #rsi_value = rsi_mat[i]-40
    #rsi_value = rsi_value/100   
    
    rsi_value = rsi_mat[i]
    #Short
    if rsi_mat[i] >= 70: test.append(rsi_value)
    #Idle
    elif rsi_mat[i] > 30 and (rsi_mat[i] < 70): test.append(0)
    #Long
    elif rsi_mat[i] <= 30: test.append(rsi_value)
            
#Plot
teste_graph = go.Scatter(x = time, y = test,  name='RSI Pulse', mode = 'markers',
                        marker = dict(color = 'rgba(152, 0, 0, .8)'))

test_graph = [teste_graph, rsi_graph]


py.iplot(test_graph, filename='basic-bar')

In [254]:
#Clustering

#Try: ((RSI-EMA(RSI,50)) / 100)**2
#Data = [rsi, strategy, ???]
array = np.zeros([len(rsi), 3], dtype=np.double)

for i in range(len(rsi)):   

    array[i,0] = rsi[i]
    array[i,1] = test[i]
    
    if rsi[i] >= 80: array[i,2] = -1
    elif rsi[i] > 20 and rsi[i] < 80: array[i,2]= 0
    elif rsi[i] <= 20: array[i,2]= 1
    

cluster = finance.KMeans(3, array)
labels = cluster.labels_

#Plot
cluster_graph = go.Scatter(x = time, y = ema, mode = 'markers', 
                           marker=dict(color = labels, colorscale='Viridis'))
data = [cluster_graph]
fig = go.Figure(data=data)
py.iplot(fig, filename='simple-3d-scatter')



KeyError: 0